# 메시지 자르기 (Trim Messages)

이 노트북에서는 **대화가 길어질 때 메시지를 자르는** 방법을 배웁니다.

## 왜 메시지를 잘라야 할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    토큰 제한 문제                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   LLM의 한계:                                                      │
│   • 한 번에 처리할 수 있는 토큰 수가 제한됨                         │
│   • GPT-4: ~128K 토큰, Llama: ~8K 토큰                             │
│   • 대화가 길어지면 한도 초과!                                      │
│                                                                    │
│   해결책:                                                          │
│   • 오래된 메시지 삭제                                              │
│   • 최근 메시지만 유지                                              │
│   • 시스템 메시지는 항상 유지                                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## trim_messages의 전략

```
┌────────────────────────────────────────────────────────────────────┐
│                 trim_messages 전략 비교                            │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   strategy='last' (기본):                                          │
│   [시스템] [메시지1] [메시지2] [메시지3] [메시지4] [메시지5]        │
│              ↓         ↓                                          │
│            삭제       삭제                                         │
│   결과: [시스템] [메시지3] [메시지4] [메시지5]                      │
│   → 최근 메시지 유지                                               │
│                                                                    │
│   strategy='first':                                                │
│   [시스템] [메시지1] [메시지2] [메시지3] [메시지4] [메시지5]        │
│                                         ↓         ↓               │
│                                       삭제       삭제              │
│   결과: [시스템] [메시지1] [메시지2] [메시지3]                      │
│   → 초기 메시지 유지                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama tiktoken

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 샘플 메시지 준비

긴 대화 기록을 만들어봅시다.

In [ ]:
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
)

# 긴 대화 기록 (11개 메시지)
messages = [
    SystemMessage(content='당신은 친절한 어시스턴트입니다.'),
    HumanMessage(content='안녕하세요! 나는 민혁입니다.'),
    AIMessage(content='안녕하세요!'),
    HumanMessage(content='바닐라 아이스크림을 좋아해요.'),
    AIMessage(content='좋네요!'),
    HumanMessage(content='2 + 2는 얼마죠?'),
    AIMessage(content='4입니다.'),
    HumanMessage(content='고마워요.'),
    AIMessage(content='천만에요!'),
    HumanMessage(content='즐거운가요?'),
    AIMessage(content='예!'),
]

print(f"총 메시지 수: {len(messages)}개\n")
print("=== 전체 대화 기록 ===")
for i, msg in enumerate(messages):
    role = msg.type
    print(f"[{i+1}] {role}: {msg.content}")

# 3. trim_messages 사용하기

## 주요 옵션 설명

| 옵션 | 설명 | 예시 |
|------|------|------|
| **max_tokens** | 최대 토큰 수 | 65 |
| **strategy** | 자르기 전략 | 'last' (최근 유지) |
| **token_counter** | 토큰 계산 모델 | LLM 모델 |
| **include_system** | 시스템 메시지 포함 | True |
| **allow_partial** | 메시지 부분 자르기 허용 | False |
| **start_on** | 시작 메시지 유형 | 'human' |

In [ ]:
from langchain_core.messages import trim_messages
from langchain_ollama import ChatOllama

# LLM 모델 (토큰 계산용)
model = ChatOllama(model='llama3.2')

# trimmer 설정
trimmer = trim_messages(
    max_tokens=65,              # 최대 65 토큰만 유지
    strategy='last',            # 최근 메시지 우선 유지
    token_counter=model,        # 토큰 계산에 사용할 모델
    include_system=True,        # 시스템 메시지는 항상 포함
    allow_partial=False,        # 메시지를 중간에 자르지 않음
    start_on='human',           # human 메시지로 시작
)

print("✅ trimmer 설정 완료")
print("   - 최대 65 토큰")
print("   - 최근 메시지 유지 전략")
print("   - 시스템 메시지 항상 포함")

# 4. 메시지 자르기 실행

In [ ]:
# trim 적용
trimmed = trimmer.invoke(messages)

print(f"원본 메시지 수: {len(messages)}개")
print(f"자른 후 메시지 수: {len(trimmed)}개\n")

print("=== 자른 후 대화 기록 ===")
for i, msg in enumerate(trimmed):
    role = msg.type
    print(f"[{i+1}] {role}: {msg.content}")

# 5. 다양한 설정 테스트

In [ ]:
# 더 많은 토큰 허용
trimmer_large = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=model,
    include_system=True,
)

trimmed_large = trimmer_large.invoke(messages)

print("=== max_tokens=100 ===")
print(f"메시지 수: {len(trimmed_large)}개")
for msg in trimmed_large:
    print(f"  {msg.type}: {msg.content}")

In [ ]:
# strategy='first' 테스트 (초기 메시지 유지)
trimmer_first = trim_messages(
    max_tokens=65,
    strategy='first',           # 초기 메시지 유지
    token_counter=model,
    include_system=True,
)

trimmed_first = trimmer_first.invoke(messages)

print("\n=== strategy='first' ===")
print(f"메시지 수: {len(trimmed_first)}개")
for msg in trimmed_first:
    print(f"  {msg.type}: {msg.content}")

# 6. 체인에서 trimmer 사용하기

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# trimmer를 체인의 일부로 사용
prompt = ChatPromptTemplate.from_messages([
    ('placeholder', '{messages}'),
])

# 체인: trimmer → prompt → model
chain = trimmer | prompt | model

print("✅ trimmer가 포함된 체인 생성 완료")

In [ ]:
# 체인 실행
result = chain.invoke(messages)

print("=== 체인 실행 결과 ===")
print(f"AI 응답: {result.content}")

---

## 정리: 메시지 자르기 (trim_messages)

### 주요 옵션

| 옵션 | 기본값 | 설명 |
|------|--------|------|
| **max_tokens** | 필수 | 유지할 최대 토큰 수 |
| **strategy** | 'last' | 'last': 최근 유지, 'first': 초기 유지 |
| **token_counter** | 필수 | 토큰 계산에 사용할 모델/함수 |
| **include_system** | False | 시스템 메시지 항상 포함 여부 |
| **allow_partial** | False | 메시지 중간 자르기 허용 |
| **start_on** | None | 시작해야 할 메시지 유형 |

### 핵심 코드

```python
from langchain_core.messages import trim_messages

# 1. trimmer 생성
trimmer = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=model,
    include_system=True,
)

# 2. 메시지 자르기
trimmed = trimmer.invoke(messages)

# 3. 체인에서 사용
chain = trimmer | prompt | model
```

### 전략 선택 가이드

| 상황 | 추천 전략 |
|------|----------|
| 일반 대화 | `strategy='last'` (최근 맥락 중요) |
| 문서 요약 | `strategy='first'` (초기 내용 중요) |
| 시스템 지시 중요 | `include_system=True` |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
token_counter = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
token_counter = ChatOllama(model='llama3.2')
```

## 다음 단계

**메시지를 유형별로 필터링**하는 방법을 배웁니다. (12-14번 노트북)